In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

In [3]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
print(f"n_user={n_user}, n_item={n_item}")
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

n_user=940, n_item=1447


In [4]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"       : evaluators.recall,
    #"Unpopularity" : evaluators.unpopularity,
    #"Unpopularity2": evaluators.unpopularity2,
    "Unpopularity3": evaluators.unpopularity3,
    #"F1-score"     : evaluators.f1_score,
    #"F1-score2"    : evaluators.f1_score2,
    "F1-score3"    : evaluators.f1_score3
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [5]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [7]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 10
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = False
search_range = 30

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)
mymp = searches.Mymp(model, search_range)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [8]:
trainer.fit(n_batch, n_epoch, knn, userwise, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:15<00:00, 61.08it/s]


In [9]:
re_knn = trainer.valid_scores.copy()
display(re_knn)

,Recall@5,Unpopularity3@5,F1-score3@5,Recall@10,Unpopularity3@10,F1-score3@10,Recall@50,Unpopularity3@50,F1-score3@50,epoch,losses
0,0.062271,0.061371,0.029432,0.124960,0.123999,0.089361,0.522693,0.525608,0.515630,0,NaN
0,0.219088,0.017148,0.020699,0.393568,0.040768,0.061940,0.817736,0.418932,0.478737,10,0.456192


In [ ]:
trainer.fit(n_batch, n_epoch, mp, userwise, valid_per_epoch)

In [ ]:
re_mp = trainer.valid_scores.copy()
display(re_mp)